In [ ]:
# Block 1: Ultra-Fast IBM Quantum VQC - Sub-10-Minute Execution
import time
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("ULTRA-FAST QUANTUM ML ON IBM HARDWARE")
print("=" * 45)

# IBM Quantum Connection
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="0w5FimduA1p4i5Y6CsyzgWmy8qvSJSSl0iJiPujlK8vA",
    overwrite=True
)
service = QiskitRuntimeService(
    channel="ibm_quantum_platform",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/6299380f8f674b28aae24826c991b3a5:9a9befce-c6b7-4020-94b1-ab38cd746cc2::"
)
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=3)
sampler = Sampler(backend)
print(f"Backend: {backend.name} ({backend.status().pending_jobs} queued)")

# Minimal Data Prep
selector = SelectKBest(score_func=f_classif, k=3)
X_selected = selector.fit_transform(X_train, y_train)[:20]  # Only 20 samples!
y_selected = y_train.iloc[:20].values
if len(np.unique(y_selected)) > 2:
    y_selected = (y_selected == y_selected[0]).astype(int)
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_quantum = scaler.fit_transform(X_selected)
print(f"Dataset: {len(X_quantum)} samples, 3 features, 2 classes")

# Minimal Quantum Circuits
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
feature_map = ZZFeatureMap(feature_dimension=3, reps=1, entanglement='linear')
ansatz = RealAmplitudes(num_qubits=3, reps=1, entanglement='linear')
optimizer = COBYLA(maxiter=10)
print(f"Circuits: FM(1 rep), Ansatz({ansatz.num_parameters} params), Opt(10 iter)")

# Hardware Transpilation
pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
feature_map_hw = pm.run(feature_map)
ansatz_hw = pm.run(ansatz)
initial_params = np.random.uniform(0, 2*np.pi, ansatz_hw.num_parameters)
print(f"Transpiled depth: {feature_map_hw.depth() + ansatz_hw.depth()}")

# Quantum Training
from qiskit_machine_learning.algorithms.classifiers import VQC
vqc = VQC(
    sampler=sampler,
    feature_map=feature_map_hw,
    ansatz=ansatz_hw,
    optimizer=optimizer,
    initial_point=initial_params
)
start_total = time.time()
train_start = time.time()
vqc.fit(X_quantum, y_selected)
train_time = time.time() - train_start
print(f"Training completed: {train_time:.1f}s")

# Quick Evaluation
print("6. Evaluation...")
y_pred = vqc.predict(X_quantum[:10])  # Only 10 test samples
accuracy = accuracy_score(y_selected[:10], y_pred)
total_time = time.time() - start_total
print(f"\nRESULTS:")
print(f"Accuracy: {accuracy:.3f}")
print(f"Training time: {train_time:.1f}s")
print(f"Total time: {total_time:.1f}s")
if total_time < 600:
    print("SUCCESS: Completed within 10-minute limit!")
else:
    print("WARNING: Exceeded 10-minute limit")
quantum_success = True
vqc_accuracy = accuracy


In [ ]:
# Block 2: Quantum Machine Learning (Top 3 Features) - Ultra-Fast Parameters
import time
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("PART V: QUANTUM MACHINE LEARNING (TOP 3 FEATURES) - ULTRA-FAST")
print("=" * 60)

# IBM Quantum Connection (repeat for self-containment)
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="0w5FimduA1p4i5Y6CsyzgWmy8qvSJSSl0iJiPujlK8vA",
    overwrite=True
)
service = QiskitRuntimeService(
    channel="ibm_quantum_platform",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/6299380f8f674b28aae24826c991b3a5:9a9befce-c6b7-4020-94b1-ab38cd746cc2::"
)
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=3)
sampler = Sampler(backend)
print(f"Backend: {backend.name} ({backend.status().pending_jobs} queued)")

# Top 3 Feature Selection, Minimal Data
selector = SelectKBest(score_func=f_classif, k=3)
X_train_selected = selector.fit_transform(X_train, y_train)[:20]
y_train_selected = y_train.iloc[:20].values
X_test_selected = selector.transform(X_test)[:10]
y_test_selected = y_test.iloc[:10].values

selected_features = X_train.columns[selector.get_support()]
feature_scores = selector.scores_[selector.get_support()]
sorted_indices = np.argsort(feature_scores)[::-1]
selected_features = selected_features[sorted_indices]
feature_scores = feature_scores[sorted_indices]

print(f"  ✓ Selected top 3 features (deterministic order):")
for i, (feature, score) in enumerate(zip(selected_features, feature_scores)):
    print(f"    {i+1}. {feature}: {score:.3f}")

X_train_selected = X_train_selected[:, sorted_indices]
X_test_selected = X_test_selected[:, sorted_indices]

scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)
print(f"  ✓ Features scaled to [0, π] range")
print(f"  Feature dimensions: {X_train_scaled.shape[1]}")

# Minimal Quantum Circuit and Optimizer
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
num_features = X_train_scaled.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1, entanglement='linear')
ansatz = RealAmplitudes(num_qubits=num_features, reps=1, entanglement='linear')
optimizer = COBYLA(maxiter=10)
pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
feature_map_hw = pm.run(feature_map)
ansatz_hw = pm.run(ansatz)
initial_params = np.random.uniform(0, 2*np.pi, ansatz_hw.num_parameters)

from qiskit_machine_learning.algorithms.classifiers import VQC
vqc = VQC(
    sampler=sampler,
    feature_map=feature_map_hw,
    ansatz=ansatz_hw,
    optimizer=optimizer,
    initial_point=initial_params
)
print("\nTraining VQC (ultra-fast, top 3 features)...")
train_start = time.time()
vqc.fit(X_train_scaled, y_train_selected)
train_time = time.time() - train_start
print(f"Training completed: {train_time:.1f}s")

y_pred_vqc = vqc.predict(X_test_scaled)
vqc_accuracy = accuracy_score(y_test_selected, y_pred_vqc)
vqc_precision = precision_score(y_test_selected, y_pred_vqc, average='weighted', zero_division=0)
vqc_recall = recall_score(y_test_selected, y_pred_vqc, average='weighted', zero_division=0)

print(f"\n✓ VQC (ultra-fast) results:")
print(f"  Accuracy:  {vqc_accuracy:.6f}")
print(f"  Precision: {vqc_precision:.6f}")
print(f"  Recall:    {vqc_recall:.6f}")
print(f"  Features used: {list(selected_features)}")

cm = confusion_matrix(y_test_selected, y_pred_vqc)
print(f"\nConfusion Matrix:")
print(cm)
print(f"\nClassification Report:")
print(classification_report(y_test_selected, y_pred_vqc, zero_division=0))
quantum_success = True
